### Fine_Tuned_Custom_Text_Generation_Model(Custome_LLM)

### Import the libraries

In [8]:
# -Import the required libraries 
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import TFBertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split
import pickle

In [4]:

# -Load your pre-trained custom language model-
custom_model = tf.keras.models.load_model('custom_llm_model.h5')

# Load your custom dataset from a CSV file-
dataset_path = 'fine_tuning_dataset.csv' 
df = pd.read_csv(dataset_path)

# -Combine 'Text' and 'Description' columns into a single corpus-
corpus = df['Text'] + " " + df['Description']

# -Tokenize the text data-
vocab_size = 50000  
max_seq_length = 50  

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(corpus)

# -Convert text to sequences-
X_sequences = tokenizer.texts_to_sequences(corpus)

# -Create training sequences (X_train) and labels (y_train) for text generation-
sequences = []
for seq in X_sequences:
    for i in range(1, len(seq)):
        sequences.append(seq[:i+1])

# -Pad sequences-
X_padded = pad_sequences(sequences, maxlen=max_seq_length, padding='pre', truncating='pre')
y_train = X_padded[:, -1]
X_train = X_padded[:, :-1]

# -Compile and train the model-
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Adjusting the learning rate
custom_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
custom_model.fit(X_train, y_train, epochs=10, batch_size=32)


2023-09-16 01:37:44.293494: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-09-16 01:37:44.965803: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-16 01:37:44.968565: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-16 01:37:44.9

Epoch 1/10


2023-09-16 01:37:47.307425: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-16 01:37:47.310640: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-16 01:37:47.313593: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1784/1784 [==============================] - 294s 162ms/step - loss: 6.6714 - accuracy: 0.1171
Epoch 2/10
1784/1784 [==============================] - 310s 174ms/step - loss: 5.4071 - accuracy: 0.1831
Epoch 3/10
1784/1784 [==============================] - 315s 177ms/step - loss: 4.5830 - accuracy: 0.2305
Epoch 4/10
1784/1784 [==============================] - 304s 171ms/step - loss: 3.7905 - accuracy: 0.2960
Epoch 5/10
1784/1784 [==============================] - 320s 179ms/step - loss: 3.0612 - accuracy: 0.3937
Epoch 6/10
1784/1784 [==============================] - 322s 180ms/step - loss: 2.4284 - accuracy: 0.5011
Epoch 7/10
1784/1784 [==============================] - 320s 180ms/step - loss: 1.8938 - accuracy: 0.6076
Epoch 8/10
1784/1784 [==============================] - 306s 171ms/step - loss: 1.4580 - accuracy: 0.7015
Epoch 9/10
1784/1784 [==============================] - 301s 169ms/step - loss: 1.0944 - accuracy: 0.7837
Epoch 10/10
1784/1784 [==============================] - 

### Save the tokenizer using pickle

In [10]:
# Save the tokenizer to a file using pickle
with open('fine_tuned_tokenizer.pickle', 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)

### Save the fine_tuned model

In [11]:
# -Save the fine-tuned model-
custom_model.save('fine_tuned_custom_model.h5')